In [27]:
api_key='RGAPI-473c5960-3ed2-4d30-822d-3c0861908f09'
import requests
import time
import pandas as pd

In [28]:
root= 'https://euw1.api.riotgames.com/'
challenger_root = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
grandmaster_root = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
master_root = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

challenger_response = requests.get(root+challenger_root+'?api_key='+api_key)
grandmaster_response = requests.get(root+grandmaster_root+'?api_key='+api_key)
master_response = requests.get(root+master_root+'?api_key='+api_key)

challenger_df = pd.DataFrame(challenger_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
grandmaster_df=pd.DataFrame(grandmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
master_df=pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)


#concatenating the dataframes
high_rank_ladder=pd.concat([challenger_df, grandmaster_df, master_df])[0:10].reset_index(drop=True)
high_rank_ladder=high_rank_ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
high_rank_ladder['rank']+=1

high_rank_ladder

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,5NzokD2VZC4DB3qBJdl6LOO3PiRQcORDRHZRLgSKdBtt0YA,1580,228,141,True,False,False,False
1,2,KFCNaCBCxDEfbGcW3ltdr_of4XStG8Q1CLDY6OOFTcm1Kz0,1539,261,173,True,False,False,True
2,3,JGhpg_EeM7jdF-U90nyputqFS-hC3ftrDN---vtbuvo7uDzA,1491,239,162,True,False,False,False
3,4,U4_ndlbviSSK2g20hBy5NNOG1awHFhM2iV_u4NlV73ayAd...,1454,451,358,True,False,False,False
4,5,9lgPXUXCgFimXrAUcRSIajMSS5yweHgAliPr-_H_CumwWGY,1412,155,74,True,False,False,False
5,6,vQzSNpeoh6jaPo9lLuDWgedrEbhSqYndNE6t5EsIF2mOaj8,1353,380,306,True,False,False,True
6,7,jOdw_LVxNtdTh65T_VL7qRzrguVEKKkqQt9CUyL0f7fMdDI,1334,334,255,True,False,False,False
7,8,jzy165pB1MR9apJ8j-jvwObhRyI7S0h9MiVgyu5-45F24qLL,1322,313,236,True,False,False,False
8,9,9jvlQ8o0So3MXFOflZbZ8z1BmeGpS1dQJyWxxjUpFy8l2dY,1303,369,286,True,False,False,False
9,10,MGgXK2GCsZNiKcQtLbFaegrbPvvjOr6YOPJTr1k-3MFGUVg,1286,166,90,True,False,False,False


In [29]:
def get_puuid_from_summoner_id(summonerId=None):
    link=f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}'

    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        puuid=response.json()['puuid']
        return puuid

In [30]:
puuid_list = []

for summoner in high_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    puuid_list.append(puuids)

In [31]:
def get_match_id_from_puuid(puuid=None):
    """
    Function to retrieve the most recent 20 match id's. Takes a player's puuid as argument and outputs the match id's of their most recent 20 ranked matches.
    Queue and type must remain as 420 and ranked respectively as this will only get the ranked matches which is what we want.
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=15&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


In [32]:
all_match_id_list=[]

for player_puuid in puuid_list:
    match_ids=get_match_id_from_puuid(puuid=player_puuid)
    all_match_id_list.extend(match_ids)

In [33]:
def get_match_info_at_twenty(matchId=None):
    link = link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('Rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data = response.json()

    if match_data['info']['queueId'] ==420:
        timeline_api_link = f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key={api_key}'

        while True:
            timeline_response = requests.get(timeline_api_link)
            if timeline_response.status_code==429:
                print('Rate limit exceeded')
                time.sleep(10)
                continue
            elif timeline_response.status_code==200:
                break
            elif timeline_response.status_code==404:
                return 0
            else:
                print('Response is bad: '+str(timeline_response.status_code))
        timeline_info = timeline_response.json()
        return timeline_info
    
    else:
        print('not a ranked match')
        return 0
            


In [59]:
get_match_info_at_twenty(matchId='EUW1_6920157556')

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6920157556',
  'participants': ['LODwZRko-7nG3YP52Ut38VLLikjOGjn9ptUF3Xe9IE-ZcHlaCfvVFNVc51QD8ai9f8E0DPSJzVXsWg',
   'buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
   'GMVUnams9_7ayqy40s8ge-gLFn9IrLHg1rDMjLBNEo7PfjREXBDESovoeKmzmspiRtRrB_c5EmgR1w',
   'JHGAIrTdVdOuqFm7TxwAhVbv-ulrxKyUtSdWd2Qk-AksgOE8PH-iknd_yZ3fmuRn-sVyTDx-KiewPQ',
   'LfhPCOUD82G9-Oeasi8DcALxFueXdKSKt0ywW4p6Sve0WThXH2sRkkn2_Qaa31ekWp-LNXKFZsjGUw',
   '3JGaafusdvWPbzEeRXswR9bRjtzpmhXkQ2fqB8l_37CRm334FStezVEZu7yqNq-cucYGf1-jOzcERg',
   '_oQzUeiMXMpnnTWeRP0nckU4lK5HFvhd4lOOCz7rHoIHMKZ3XkILdze0Kio51feikjpa9ZVXn5NuSw',
   '2f9TQdqO4CD41tRmwuHEdpqwY18rhyqzwOjf6GfBjEDJyDCEXG4HJbS_qHRbweXoqkJY6YnZi2qMBQ',
   'qkROYMEq2-V-f_50WgZ7y5x35kQlm4W2IFVf3sgTsShb6Z6ngUdBkS7BLMZP4VCLU7H34H8RWiJjHg',
   'zLqSKKcgzM2h66_dhlnwnBnpBVSfzLYIEV6o5ou2Onie4wUTlwvf4JsO_PCJvU-aj_96lRNm-jPA-g']},
 'info': {'endOfGameResult': 'GameComplete',
  'frameInterval': 60

In [35]:
import pandas as pd

In [102]:
def get_specific_match_stats_at_twenty(data):
    blue_side = data['info']['participants'][:5]
    red_side = data['info']['participants'][5:]

    #blue side statistics
    #we will not consider baron in this because it spawns at exactly 20 minutes so it's impossible for any team to kill it in time
    total_blue_side_minions = 0
    total_blue_side_kills=0
    total_blue_side_deaths=0
    total_blue_side_gold_earned = 0
    total_blue_side_wards_placed = 0
    total_towers_destroyed_by_blue = 0
    total_inhibitors_destroyed_by_blue =0
    total_blue_side_dragons_killed =0
    total_blue_side_riftHeralds_killed =0
    blue_team_first_blood =0
    blue_side_won=0

    #red side statistics
    total_red_side_minions = 0
    total_red_side_kills=0
    total_red_side_deaths =0
    total_red_side_gold_earned = 0
    total_red_side_wards_placed = 0
    total_towers_destroyed_by_red = 0
    total_inhibitors_destroyed_by_red =0
    total_red_side_dragons_killed =0
    total_red_side_riftHeralds_killed =0
 


    if len(data['info']['frames']) < 21:
        print('Game is shorter than 20 minutes')
    
    #else:
    #getting if blue side won or not (target variable, 1= blue, 0 = red)    
    for event in data['info']['frames'][-1]['events']:
        if event['type'] == 'GAME_END':
            if event['winningTeam'] ==100:
                blue_side_won+=1
    
    for participant in blue_side:
        total_blue_side_minions += data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['minionsKilled']
        total_blue_side_gold_earned+= data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['totalGold']

    for participant in red_side:
        total_red_side_minions += data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['minionsKilled']
        total_red_side_gold_earned+= data['info']['frames'][20]['participantFrames'][str(participant['participantId'])]['totalGold']


    #for loop to iterate over the events that occurred at the first 20 minutes/'frames' in the game
    for frame in range(20):
        for event in data['info']['frames'][frame]['events']:
            if event['type']=='WARD_PLACED':
                if event['creatorId'] in range(1,6):
                    total_blue_side_wards_placed+=1
                else:
                    total_red_side_wards_placed+=1

            #kill stats for both teams
            if event['type']=='CHAMPION_KILL':
                if event['killerId'] in range(1,6):
                    total_blue_side_kills+=1
                    total_red_side_deaths+=1
                else:
                    total_red_side_kills+=1
                    total_blue_side_deaths+=1
            
            #objective stats for both teams
            if event['type']=='BUILDING_KILL' and event['buildingType'] == 'TOWER_BUILDING':
                if event['killerId'] in range (1,6):
                    total_towers_destroyed_by_blue+=1
                else:
                    total_towers_destroyed_by_red+=1
            
            #tower/turret stats
            if event['type']=='BUILDING_KILL' and event['buildingType'] == 'INHIBITOR_BUILDING':
                if event['killerId'] in range (1,6):
                    total_inhibitors_destroyed_by_blue+=1
                else:
                    total_inhibitors_destroyed_by_red+=1

            #elite monsters stats. within first 20 mins the only elite monsters that can be killed are rift heralds and dragons, baron nashor and elder dragon spawns after 20 mmins
            if event['type']=='ELITE_MONSTER_KILL':
                if event['monsterType'] == 'DRAGON':
                    if event['killerId'] in range(1,6):
                        total_blue_side_dragons_killed+=1
                    else:
                        total_red_side_dragons_killed+=1
                if event['monsterType']=='RIFTHERALD':
                    if event['killerId'] in range(1,6):
                        total_blue_side_riftHeralds_killed+=1
                    else:
                        total_red_side_riftHeralds_killed+=1
                    


    df = pd.DataFrame({
        #blue stats
        'total_blue_side_minions': [total_blue_side_minions],
        'total_blue_side_gold_earned': [total_blue_side_gold_earned],
        'total_blue_side_wards_placed':[total_blue_side_wards_placed],
        'total_blue_side_kills':[total_blue_side_kills],
        'total_blue_side_deaths':[total_blue_side_deaths],
        'total_blue_side_dragons_killed':[total_blue_side_dragons_killed],
        'total_blue_side_riftHeralds_killed':[total_blue_side_riftHeralds_killed],
        'total_towers_destroyed_by_blue': [total_towers_destroyed_by_blue],
        'total_inhibitors_destroyed_by_blue':[total_inhibitors_destroyed_by_blue],

        #red stats
        'total_red_side_minions':[total_red_side_minions],
        'total_red_side_gold_earned':[total_red_side_gold_earned],
        'total_red_side_kills':[total_red_side_kills],
        'total_red_side_deaths':[total_red_side_deaths],
        'total_red_side_wards_placed':[total_red_side_wards_placed],        
        'total_red_side_dragons_killed':[total_red_side_dragons_killed],
        'total_red_side_riftHeralds_killed':[total_red_side_riftHeralds_killed],
        'total_towers_destroyed_by_red':[total_towers_destroyed_by_red],
        'total_inhibitors_destroyed_by_red':[total_inhibitors_destroyed_by_red],

        'blue_side_won': [blue_side_won]


    })

    return df
        



In [72]:
all_match_id_list

['EUW1_6935074839',
 'EUW1_6934156795',
 'EUW1_6933165219',
 'EUW1_6931925745',
 'EUW1_6928661440',
 'EUW1_6928550011',
 'EUW1_6928471250',
 'EUW1_6927403998',
 'EUW1_6926350115',
 'EUW1_6925234085',
 'EUW1_6923994009',
 'EUW1_6922507359',
 'EUW1_6921497820',
 'EUW1_6920157556',
 'EUW1_6915678643',
 'EUW1_6933190928',
 'EUW1_6933144678',
 'EUW1_6929512909',
 'EUW1_6929486741',
 'EUW1_6928779209',
 'EUW1_6928512060',
 'EUW1_6928291774',
 'EUW1_6923501139',
 'EUW1_6923418997',
 'EUW1_6921387114',
 'EUW1_6915740520',
 'EUW1_6915618334',
 'EUW1_6915457903',
 'EUW1_6915394465',
 'EUW1_6914335508',
 'EUW1_6933746977',
 'EUW1_6933703853',
 'EUW1_6933648833',
 'EUW1_6933216884',
 'EUW1_6931605313',
 'EUW1_6929648620',
 'EUW1_6929604501',
 'EUW1_6926231476',
 'EUW1_6924112572',
 'EUW1_6924050763',
 'EUW1_6923820363',
 'EUW1_6923685124',
 'EUW1_6920058129',
 'EUW1_6919935575',
 'EUW1_6919839293',
 'EUW1_6934392785',
 'EUW1_6934327030',
 'EUW1_6934294226',
 'EUW1_6934254219',
 'EUW1_6934190890',


In [105]:
match_ids = all_match_id_list[0:30]
dfs = pd.DataFrame()  # Initialize an empty list to hold the data frames

for match_id in match_ids:
    game = get_match_info_at_twenty(matchId=match_id)
    if len(game['info']['frames']) >=20:  # Check if the game has at least 21 frames
        match_df = get_specific_match_stats_at_twenty(game)
        dfs=pd.concat([dfs, match_df])
    else:
        print(f"Match ID {match_id} is shorter than 20 minutes. Skipping...")

Match ID EUW1_6922507359 is shorter than 20 minutes. Skipping...
Match ID EUW1_6920157556 is shorter than 20 minutes. Skipping...
Match ID EUW1_6923501139 is shorter than 20 minutes. Skipping...


In [107]:
dfs.shape

(27, 19)

In [37]:
all_match_id_list

['EUW1_6935074839',
 'EUW1_6934156795',
 'EUW1_6933165219',
 'EUW1_6931925745',
 'EUW1_6928661440',
 'EUW1_6928550011',
 'EUW1_6928471250',
 'EUW1_6927403998',
 'EUW1_6926350115',
 'EUW1_6925234085',
 'EUW1_6923994009',
 'EUW1_6922507359',
 'EUW1_6921497820',
 'EUW1_6920157556',
 'EUW1_6915678643',
 'EUW1_6933190928',
 'EUW1_6933144678',
 'EUW1_6929512909',
 'EUW1_6929486741',
 'EUW1_6928779209',
 'EUW1_6928512060',
 'EUW1_6928291774',
 'EUW1_6923501139',
 'EUW1_6923418997',
 'EUW1_6921387114',
 'EUW1_6915740520',
 'EUW1_6915618334',
 'EUW1_6915457903',
 'EUW1_6915394465',
 'EUW1_6914335508',
 'EUW1_6933746977',
 'EUW1_6933703853',
 'EUW1_6933648833',
 'EUW1_6933216884',
 'EUW1_6931605313',
 'EUW1_6929648620',
 'EUW1_6929604501',
 'EUW1_6926231476',
 'EUW1_6924112572',
 'EUW1_6924050763',
 'EUW1_6923820363',
 'EUW1_6923685124',
 'EUW1_6920058129',
 'EUW1_6919935575',
 'EUW1_6919839293',
 'EUW1_6934392785',
 'EUW1_6934327030',
 'EUW1_6934294226',
 'EUW1_6934254219',
 'EUW1_6934190890',


In [67]:
get_match_info_at_twenty(matchId='EUW1_6922507359')

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_6922507359',
  'participants': ['zuAncw-Y2V8kOFq2ooZOFPx_BuDOyLD87p269DcyXrf44FoVfvbDDNHhhTKoP5d8ar8pMw5nzyPngg',
   'V6fMOp5_6uJzRr9zzV_OcjcolGb1u6AWMihiHauRCAKzLrCt0ysQI_9gC4osyAJ8s59JACNW_6Lf1g',
   'tFHLuq5c7F5IcfA4tKABSX5af8HLgRLEjEzyGCRLzaENRAIrmwRjfHceDOCrqLDVyYt2XwykRx8Vig',
   'PLvk6LWDB_4bdKxulfvyk5GFTNaMJIthGs-yPpW7qIqlBUF52B1mjkTuW5ngk6PFSiJ8LD7Mc-aGbQ',
   'qrdGXPjGFtt9EH-EkyfCCKE-3T-eDKBeKT26KnQPIA5QSn5f1UM6fDK3kMRpj8vDML5K3FJsaRQ_cA',
   'sOCKEt2sR01YP9W28bXwTPn7gqOnZs3_q9ZZiRBrFzqggaFnxy4cv5A9xdTULMmYOGxtCNtjdft3dw',
   'buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
   'g0ZhqegHwAgkEkbP6nl2ctb0cDX344SKDFtFXQmB1HoOgGgLHZtZ3F3Qx9W6Cwq3pdMmcxTynQEUug',
   'zRqBRkxGsLavYAtZTu5u7vnb3lYfzD0YrpDUWbEHO0rd7evpNAek4109MIbNNZOMP2ARfAFEtvSmWA',
   'q6etrDTBhLZ_keTEP9IxenZzlQMsLYneHQa6qYoitTgrkOKE7JMVra9sKMaXMGOr22waH1kLXA9MOA']},
 'info': {'endOfGameResult': 'GameComplete',
  'frameInterval': 60

In [66]:
get_specific_match_stats_at_twenty(data=get_match_info_at_twenty(matchId='EUW1_6929486741'))

,total_blue_side_minions,total_blue_side_gold_earned,total_blue_side_wards_placed,total_blue_side_kills,total_blue_side_deaths,total_blue_side_dragons_killed,total_blue_side_riftHeralds_killed,total_towers_destroyed_by_blue,total_inhibitors_destroyed_by_blue,total_red_side_minions,total_red_side_gold_earned,total_red_side_kills,total_red_side_deaths,total_red_side_wards_placed,total_red_side_dragons_killed,total_red_side_riftHeralds_killed,total_towers_destroyed_by_red,total_inhibitors_destroyed_by_red,blue_side_won
0,545,38465,35,14,17,0,1,2,0,537,41471,17,14,54,2,0,4,0,0
